# QUICKSTART
#### This section runs through the API for common tasks in machine learning. Refer to the links in each section to dive deeper.

## Working with data
#### PyTorch has two primitives to work with data: torch.utils.data.DataLoader and torch.utils.data.Dataset. Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset.

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

## Importing Datasets
#### The torchvision.datasets module contains Dataset objects for many real-world vision data like CIFAR, COCO (full list here). In this tutorial, we use the FashionMNIST dataset. Every TorchVision Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.



In [4]:
# Download training dataset
training_data = datasets.FashionMNIST(
    root='data',
    train = True,
    download=True,
    transform = ToTensor()
)
#  Download test dataset
test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

26422272it [00:08, 3299194.01it/s]                              


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



29696it [00:00, 180562.31it/s]                          


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



4422656it [00:02, 2077355.00it/s]                             


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



6144it [00:00, 9453339.61it/s]          
/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1616554845587/work/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Processing...
Done!


#### We pass the Dataset as an argument to DataLoader. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [5]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
for X,y in test_dataloader:
    print(X.shape, y.shape)
    break
print(len(train_dataloader),len(test_dataloader))

torch.Size([64, 1, 28, 28]) torch.Size([64])
938 157


## Creating Models
#### To define a neural network in PyTorch, we create a class that inherits from nn.Module. We define the layers of the network in the __init__ function and specify how data will pass through the network in the forward function. To accelerate operations in the neural network, we move it to the GPU if available.

In [6]:
# Get cpu or gpu device for training.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512 ,10)
        )
    def forward(self , x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)


Using {device} device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters
#### To train a model, we need a loss function and an optimizer.



In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= 1e-3)

## Training Function
#### In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [8]:
def train(data_loader, model, loss_fn, optimizer):
    size = len(data_loader)
    model.train()
    for batch, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)

        pred = model(X) 
        loss = loss_fn(pred,y) #calculate loss between predicted and actual labels

        optimizer.zero_grad() # initialize gradients to zeros
        loss.backward() # backpropagate loss
        optimizer.step() # Update the parameters

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

## Test Function
#### We also check the model’s performance against the test dataset to ensure it is learning.

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # print(f"Size {size} and batches {num_batches} =")
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            # print(pred.argmax(0),'--', y)
    # print(f"test_loss {test_loss} and correct {correct}")
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.243749  [    0/  938]
loss: 2.230036  [ 6400/  938]
loss: 2.199656  [12800/  938]
loss: 2.204700  [19200/  938]
loss: 2.170856  [25600/  938]
loss: 2.136001  [32000/  938]
loss: 2.150854  [38400/  938]
loss: 2.098703  [44800/  938]
loss: 2.102810  [51200/  938]
loss: 2.051353  [57600/  938]
Test Error: 
 Accuracy: 52.0%, Avg loss: 2.047704 

Epoch 2
-------------------------------
loss: 2.066748  [    0/  938]
loss: 2.041564  [ 6400/  938]
loss: 1.960200  [12800/  938]
loss: 1.990125  [19200/  938]
loss: 1.899215  [25600/  938]
loss: 1.846612  [32000/  938]
loss: 1.868581  [38400/  938]
loss: 1.770791  [44800/  938]
loss: 1.789523  [51200/  938]
loss: 1.695460  [57600/  938]
Test Error: 
 Accuracy: 59.2%, Avg loss: 1.703443 

Epoch 3
-------------------------------
loss: 1.748111  [    0/  938]
loss: 1.707175  [ 6400/  938]
loss: 1.575535  [12800/  938]
loss: 1.637174  [19200/  938]
loss: 1.510743  [25600/  938]
loss: 1.495602  [32000/  9

## Saving Models
#### A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [12]:
torch.save(model.state_dict(),'model.pth')
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Models
#### The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

#### This model can now be used to make predictions.

In [39]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[1000][0], test_data[1000][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

tensor(6) 0
Predicted: "Shirt", Actual: "T-shirt/top"
